In [3]:
%load_ext autoreload
%autoreload 2
import requests
from configurations.models import models
from models.dumb.dumb_interviewer import DumbInterviewer

def get_cutoff(from_experiment):
    return {'separation': 3}.get(from_experiment, 10)

def print_missing(missing):
    if missing:
        for idx, name in enumerate(sorted(missing)):
            print(f'{idx + 1}. {name}')
    else:
        print('Nothing!')
    print()

model_names = set()
# Change prefix to e.g., greedy, ddpg, to show only those interviewers
experiment = 'separation'
model_prefix = ''
num_questions = 10
splits = 4

data = requests.get(f'https://mindreader.tech/spectate/results/{experiment}/ndcg/{get_cutoff(experiment)}').json()
data = {k: v for k, v in data.items() if not 'grid' in k}

for model in models:
    if not model.startswith(model_prefix):
        continue
    
    model_names.add(model)
    
    if models[model]['interviewer'] != DumbInterviewer:
       model_names.add(f'{model}-rec')

existing_models = set(data.keys())

print(f'Missing entirely from {experiment} ({model_prefix}):')
print_missing(model_names.difference(existing_models))

print(f'Missing first split:')
print_missing({model for model in existing_models if len(data[model]) < num_questions})

print(f'Missing splits:')
def results_len(from_model):
    return sum([len(split) for split in [question for question in data[from_model]]])

print_missing({model for model in existing_models if len(data[model]) == num_questions and results_len(model) < num_questions * splits})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Missing entirely from separation ():
1. ddpg-ppr-collab-rec
2. ddpg-ppr-kg-rec
3. dqn-ppr-collab
4. dqn-ppr-collab-rec
5. dqn-ppr-kg
6. dqn-ppr-kg-rec

Missing first split:
1. ddpg-mf
2. ddpg-mf-rec
3. ddpg-ppr-collab

Missing splits:
1. ddpg-knn
2. ddpg-knn-rec
3. ddpg-ppr-joint
4. ddpg-ppr-joint-rec
5. fmf
6. fmf-rec

